<a href="https://colab.research.google.com/github/Waleed-Mujahid/CAO-project/blob/main/Cao_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# including dependecies
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import keras

In [ ]:
# image dimensions, set as per need
img_width, img_height = 96,96  # 96 x 96 suitable for micro-controller

In [ ]:
# Loading our dataset from Drive
# First mount drive

train_data_dir = '/content/drive/MyDrive/cao data/train'
validation_data_dir = '/content/drive/MyDrive/cao data/test'
test_data_dir = '/content/drive/MyDrive/test_Data'

In [ ]:
# Data pre-processing
train_datagen = ImageDataGenerator( 
rescale=1. / 255,
rotation_range = 10,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

In [ ]:
# rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Getting data generator from directory

train_generator = train_datagen.flow_from_directory(
train_data_dir,
batch_size = 8,
target_size=(img_width, img_height),
color_mode="grayscale",
class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
batch_size = 8,
color_mode="grayscale",
target_size=(img_width, img_height),
class_mode='binary')

test_generator = test_datagen.flow_from_directory(
test_data_dir,
batch_size = 8,
color_mode="grayscale",
target_size=(img_width, img_height),
class_mode='binary')

In [ ]:
# Building our model in Keras

model = Sequential()
model.add(Conv2D(4, (5, 5), strides = (2,2), input_shape=(img_width, img_height,1)))  # Input size of all images must be standard for model
model.add(MaxPooling2D(strides=2))

model.add(Conv2D(8, (4, 4), strides = (2,2)))
model.add(MaxPooling2D(strides=2))
model.add(keras.layers.Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(MaxPooling2D(strides=2))
model.add(keras.layers.Activation('relu'))

model.add(Flatten())
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
metrics=['accuracy'])

In [ ]:
history = model.fit(
train_generator,
epochs=10,
validation_data=validation_generator,
batch_size = 8,
class_weight= {0: 1,1: 3} # weight bcs ou data set is imballanced
) 

In [ ]:
model.evaluate(test_generator)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('cao.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
#!apt-get -qq install xxd
!xxd -i cao.tflite > model.cc